<a href="https://colab.research.google.com/github/park-geun-hyeong/AI_service_from_youtube/blob/main/Text_Rogistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unzip file

In [4]:
import os 

path = '/content/drive/MyDrive/PyTorch/sentiment'
os.chdir(path)

In [ ]:
os.listdir(path)

['sentiment.ipynb', 'aclImdb_v1.tar.gz']

In [ ]:
!tar xvzf  /content/drive/MyDrive/PyTorch/sentiment/aclImdb_v1.tar.gz

In [ ]:
! pip install pyprind

In [2]:
import pandas as pd
import numpy as np

In [ ]:
basepath = '/content/drive/MyDrive/PyTorch/sentiment/aclImdb'

## from txt file to csv file

In [ ]:
labels = {'neg':0,'pos':1}
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()
for s in ('train','test'):
    for i in ('neg','pos'):
        path = os.path.join(basepath,s,i)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
                txt = infile.read()
            df=df.append([[txt,labels[i]]],ignore_index=True)
            pbar.update()

df.columns=['text','sentiment']    
                

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:47


In [ ]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.to_csv('sentiment.csv',index=False,encoding='utf-8')

In [5]:
df = pd.read_csv('sentiment.csv',encoding='utf-8')

## Data

In [6]:
df.head()

,text,sentiment
0,Watching a videotaped replay of about 8 variou...,0
1,It's easy to make really general comments abou...,1
2,Reading the comments I am struck by the obviou...,1
3,Frank Tashlin's 'Censored' is a so-so Private ...,0
4,"GREAT, Chris Diamantopoulos has got to be the ...",1


In [7]:
df.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

## Preprocessing Text

In [8]:
from bs4 import BeautifulSoup
import re

In [9]:
! pip install nltk

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [12]:
def preprocessing(sentence):
    text = sentence.lower()
    text = BeautifulSoup(text,'html.parser')
    text = re.sub('[^A-Za-z]',' ',text.text) 
    text = [a for a in text.split() if a not in stopwords]

    return ' '.join(text)

In [13]:
texts = df.text

In [14]:
cleaned_texts = texts.apply(preprocessing)

In [15]:
df['text'] = cleaned_texts

In [16]:
df

,text,sentiment
0,watching videotaped replay various spider man ...,0
1,easy make really general comments film like fa...,1
2,reading comments struck obvious effect wonderf...,1
3,frank tashlin censored private snafu short aim...,0
4,great chris diamantopoulos got best robim will...,1
...,...,...
49995,ho ho homicidal maniac spirited tour de force ...,1
49996,opening shot consequences love perfectly sets ...,1
49997,alice enters world wonder kind wonder remember...,1
49998,breezy highly entertaining drama excellent cas...,1


In [17]:
df.text[0]

'watching videotaped replay various spider man cartoons made realize stomach first came old school spider man cartoon best still remains best get psychedelic version terrible give traditional villains please acting new stuff lousy read sheet either feeling overacting paul soles guy best voice acting spider man one comes close watching secret wars great idea cartoon mini series made wince dr doom sounds like comedy version bela lugosi scene red skull doc ock red skull german accent ock heavy russian old marvel comic hero series much better voice acting iron man sounded like wearing iron mask captain america sounded authoritative like teenage kid paul frees thing best thing ever old voice actors pioneers never anyone like ever hear mel blanc son way replicate dad animation clunky okay fancy character shadings nicely painted backrounds sometimes say less movement movement looks bad sometimes let computer take movements become robotic really think animators know betweeners stories badly wr

## Text Tokenizer

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
cv = CountVectorizer(analyzer='word',max_features=10000)

In [20]:
text_cv = cv.fit_transform(df.text)

In [21]:
text_cv.shape

(50000, 10000)

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer 

In [26]:
tfid = TfidfTransformer()

In [27]:
text_tfid = tfid.fit_transform(text_cv).toarray()

In [28]:
text_tfid.shape

(50000, 10000)

In [29]:
x = text_tfid

In [30]:
y = df.sentiment.values

In [31]:
x.shape, y.shape

((50000, 10000), (50000,))

In [32]:
type(x), type(y)

(numpy.ndarray, numpy.ndarray)

In [33]:
x_train = x[:25000]
y_train = y[:25000]

x_test = x[25000:]
y_test = y[25000:]

In [34]:
y_train = y_train[:,np.newaxis]
y_test = y_test[:,np.newaxis]

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [55]:
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

x_test_tensor = torch.from_numpy(x_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [91]:
x_train_tensor.shape, y_train_tensor.shape, x_test_tensor.shape, y_test_tensor.shape

(torch.Size([25000, 5000]),
 torch.Size([25000, 1]),
 torch.Size([25000, 5000]),
 torch.Size([25000, 1]))

## Model

In [39]:
model = nn.Sequential(

    nn.Linear(10000,1),
    nn.Sigmoid()
)

In [40]:
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [41]:
from tqdm import tqdm_notebook

In [42]:
import gc
gc.collect()

211

## Train

In [65]:
for epoch in tqdm_notebook(range(1000)):
    optimizer.zero_grad()

    logits = model(x_train_tensor)
    loss = F.binary_cross_entropy(logits, y_train_tensor)

    loss.backward()
    optimizer.step()

    if epoch%100==0:
        print(f'epoch:{epoch}, loss:{loss.item():.4f}')  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


epoch:0, loss:0.0910
epoch:100, loss:0.0838
epoch:200, loss:0.0774
epoch:300, loss:0.0718
epoch:400, loss:0.0667
epoch:500, loss:0.0621
epoch:600, loss:0.0579
epoch:700, loss:0.0541
epoch:800, loss:0.0506
epoch:900, loss:0.0474



In [66]:
test_pred = model(x_test_tensor)

In [67]:
prediction = test_pred >= torch.FloatTensor([0.5])


In [68]:
prediction

tensor([[ True],
        [False],
        [False],
        ...,
        [ True],
        [ True],
        [False]])

In [69]:
eval_bool = (prediction==y_test_tensor)

In [70]:
torch.mean(eval_bool.float())

tensor(0.8360)